In [1]:
import mlrun
import time

> 2024-04-01 06:17:22,626 [warning] Failed resolving version info. Ignoring and using defaults
> 2024-04-01 06:17:25,574 [warning] Server or client version is unstable. Assuming compatible: {"client_version":"0.0.0+unstable","server_version":"1.7.0-rc6"}


In [2]:
project_name = "model-serving"
project = mlrun.get_or_create_project(name=project_name)

> 2024-04-01 06:17:25,768 [info] Loading project from path: {"path":"./","project_name":"model-serving"}
> 2024-04-01 06:17:41,171 [info] Project loaded successfully: {"path":"./","project_name":"model-serving","stored_in_db":true}


In [3]:
# mlrun: start-code

In [4]:
import mlrun

In [5]:
preprocess_seconds = 0.1
prediction_seconds = 0.1
postprocess_seconds = 0.1

In [6]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        pass

    def predict(self, body: dict):
        # time.sleep(prediction_seconds)
        return []


#     def preprocess(self, request: dict, operation) -> dict:
#         time.sleep(preprocess_seconds)
#         return request

#     def postprocess(self, request: dict):
#         time.sleep(postprocess_seconds)
#         return request

In [7]:
# mlrun: end-code

In [8]:
fn = mlrun.code_to_function(
    "v2-model-server",
    kind="serving",
    code_output=".",
    image="mlrun/mlrun:1.7.0-rc6",
)
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.spec.default_class = "ClassifierModel"

In [10]:
fn.add_model("mymodel", model_path="abc")

In [ ]:
fn.apply(mlrun.mount_v3io())
invoke_url = fn.deploy(project=project_name)

> 2024-04-01 06:17:45,611 [info] Starting remote function deploy
2024-04-01 06:17:45  (info) Deploying function
2024-04-01 06:17:45  (info) Building
2024-04-01 06:17:45  (info) Staging files and preparing base images
2024-04-01 06:17:45  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-04-01 06:17:45  (info) Building processor image


In [ ]:
!wget https://github.com/gtopper/postto/releases/download/v1.2.1/postto_v1.2.1_linux_amd64

In [ ]:
!chmod u+x postto_v1.2.1_linux_amd64

In [ ]:
import os

os.environ["BENCH_INVOKE_URL"] = invoke_url
os.environ["BENCH_CONCURRENT_REQUESTS"] = "16"

In [ ]:
!for i in {1..50000}; do echo '{"inputs": []}'; done | ./postto_v1.2.1_linux_amd64 -num-concurrent-requests $BENCH_CONCURRENT_REQUESTS $BENCH_INVOKE_URL